In [361]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import setuptools
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
import datetime


In [362]:
filename = "../data/final_dataset.json"
df = pd.read_json(filename)
df.head()

print(len(df))

118714


In [363]:
year_threshold = datetime.datetime.today().year + 10
# Keep rows where the condition is true 
df = df.loc[
    (df["Price"] < 15000000) &
    (df["ConstructionYear"] <= year_threshold) | (pd.isna(df["ConstructionYear"]))&
    ~((df["GardenArea"] > 0) & (df["Garden"] == 0)) &
    ~((df["GardenArea"] > 0) & (df["Garden"] == 0)) &
    (df["ShowerCount"] < 30) &
    (df["ToiletCount"] < 50)     
]

len(df)

89063

In [364]:
df.drop(["Fireplace", "Furnished","PropertyId","Region", "Country", "SubtypeOfProperty", "Url", "MonthlyCharges", "RoomCount"], axis = 1, inplace=True)
df.dropna(subset=['Locality', 'District', "StateOfBuilding", "LivingArea"], how='all', inplace=True)
df.drop_duplicates(inplace= True)

exclude_annuity = ["annuity_monthly_amount", "annuity_without_lump_sum", "annuity_lump_sum", "homes_to_build"]
df = df[~df["TypeOfSale"].isin(exclude_annuity)]

print("There are {} rows of data".format(len(df)))

There are 83446 rows of data


In [365]:

if df["TypeOfSale"].dtype == 'O' : 
    df = df.loc[df['TypeOfSale'].isin(['residential_sale', 'residential_monthly_rent'])]
    df["TypeOfSale"] = df["TypeOfSale"].apply(lambda x : 0 if x == "residential_sale" else 1)

df["TypeOfProperty"] = df["TypeOfProperty"].apply(lambda x : 0 if x == 1 else 1)


In [366]:
# Subset the plausible value 
valid_peb_values = ['D', 'B', 'F', 'E', 'C', 'A', 'G', 'A++', 'A+', None]
df = df.loc[df["PEB"].isin(valid_peb_values)]

In [367]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values= None, strategy='constant', fill_value='Unknown')
df['PEB'] = imputer.fit_transform(df[['PEB']]).flatten()
imputer = SimpleImputer(missing_values= None, strategy='constant', fill_value='Unknown')
df['StateOfBuilding'] = imputer.fit_transform(df[['StateOfBuilding']]).flatten()


In [368]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder(categories=[['Unknown','A++', 'A+', 'A', 'B', 'C', 'D', 'E', 'F', 'G']])
df["PEB"] = ordinal_encoder.fit_transform(df[["PEB"]])


ordinal_encoder = OrdinalEncoder(categories=[['Unknown','AS_NEW','JUST_RENOVATED','GOOD','TO_RESTORE','TO_RENOVATE','TO_BE_DONE_UP']])
df["StateOfBuilding"] = ordinal_encoder.fit_transform(df[["StateOfBuilding"]])

binary_kitchen_mapping = {
    None : 0 , 
    'USA_HYPER_EQUIPPED': 1,
    'NOT_INSTALLED': 0,
    'USA_UNINSTALLED': 0,
    'SEMI_EQUIPPED': 1,
    'USA_SEMI_EQUIPPED': 1,
    'INSTALLED': 1,
    'USA_INSTALLED': 1,
    'HYPER_EQUIPPED': 1,
}

# Apply the mapping to create the new binary column
df['Kitchen'] = df['Kitchen'].map(binary_kitchen_mapping)
df[["Kitchen"]]

binary_flooding_mapping = {
    None : 0 ,
    'NON_FLOOD_ZONE': 0,
    'RECOGNIZED_N_CIRCUMSCRIBED_FLOOD_ZONE': 0,
    'RECOGNIZED_FLOOD_ZONE': 1,
    'RECOGNIZED_N_CIRCUMSCRIBED_WATERSIDE_FLOOD_ZONE': 1,
    'CIRCUMSCRIBED_FLOOD_ZONE': 1,
    'CIRCUMSCRIBED_WATERSIDE_ZONE': 1,
    'POSSIBLE_N_CIRCUMSCRIBED_FLOOD_ZONE': 1,
    'POSSIBLE_FLOOD_ZONE': 1,
    'POSSIBLE_N_CIRCUMSCRIBED_WATERSIDE_ZONE': 1,
}

# Apply the mapping to create the new binary column
df['FloodingZone'] = df['FloodingZone'].map(binary_flooding_mapping)
df[["FloodingZone"]]

df.loc[df['NumberOfFacades'] > 4, 'NumberOfFacades'] = 4


In [369]:
df.fillna({"Garden" : 0}, inplace=True)
df.fillna({"SwimmingPool" : 0}, inplace=True)
df.fillna({"Terrace" : 0}, inplace=True)
df.loc[(df["TypeOfProperty"] == 1) & (df["SurfaceOfPlot"].isna()), "SurfaceOfPlot"] = 0
df.loc[(df["Garden"] == 0) & (df["GardenArea"].isna()), "GardenArea"] = 0

df.isna().sum().to_dict()


{'BathroomCount': 4335,
 'BedroomCount': 0,
 'ConstructionYear': 19000,
 'District': 4,
 'FloodingZone': 0,
 'Garden': 0,
 'GardenArea': 0,
 'Kitchen': 0,
 'LivingArea': 7879,
 'Locality': 0,
 'NumberOfFacades': 25227,
 'PEB': 0,
 'PostalCode': 0,
 'Price': 0,
 'Province': 4,
 'ShowerCount': 32311,
 'StateOfBuilding': 0,
 'SurfaceOfPlot': 0,
 'SwimmingPool': 0,
 'Terrace': 0,
 'ToiletCount': 8791,
 'TypeOfProperty': 0,
 'TypeOfSale': 0}

In [370]:
imputer = KNNImputer(n_neighbors=2)
# List of columns to be imputed
columns_to_impute = ["StateOfBuilding", "FloodingZone", "Kitchen", "PEB", "SurfaceOfPlot", "ConstructionYear", "NumberOfFacades" ]

# Fit and transform the data
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])
for col in columns_to_impute : 
    df[col] = round(df[col])
# Display the transformed columns
print(df[columns_to_impute].head())

    StateOfBuilding  FloodingZone  Kitchen  PEB  SurfaceOfPlot  \
2               3.0           0.0      0.0  0.0            0.0   
6               3.0           0.0      0.0  6.0          130.0   
8               3.0           0.0      1.0  4.0            0.0   
11              1.0           0.0      0.0  4.0            0.0   
14              1.0           0.0      1.0  7.0            0.0   

    ConstructionYear  NumberOfFacades  
2             1969.0              2.0  
6             1920.0              3.0  
8             2008.0              2.0  
11            1972.0              2.0  
14            1994.0              2.0  


In [371]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

categorical_attributes = list(df.select_dtypes(include=['object']).columns)
numerical_attributes = list(df.select_dtypes(include=['float64', 'int64']).columns)

In [372]:
df.drop(["Locality", "District", "Province"], axis = 1, inplace= True)
df.columns



Index(['BathroomCount', 'BedroomCount', 'ConstructionYear', 'FloodingZone',
       'Garden', 'GardenArea', 'Kitchen', 'LivingArea', 'NumberOfFacades',
       'PEB', 'PostalCode', 'Price', 'ShowerCount', 'StateOfBuilding',
       'SurfaceOfPlot', 'SwimmingPool', 'Terrace', 'ToiletCount',
       'TypeOfProperty', 'TypeOfSale'],
      dtype='object')

In [373]:
df.isnull().sum()

BathroomCount        4335
BedroomCount            0
ConstructionYear        0
FloodingZone            0
Garden                  0
GardenArea              0
Kitchen                 0
LivingArea           7879
NumberOfFacades         0
PEB                     0
PostalCode              0
Price                   0
ShowerCount         32311
StateOfBuilding         0
SurfaceOfPlot           0
SwimmingPool            0
Terrace                 0
ToiletCount          8791
TypeOfProperty          0
TypeOfSale              0
dtype: int64

In [374]:
X = df.drop(columns=["Price"], axis = 1)
y = df["Price"]

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [375]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [376]:
regressor = xgb.XGBRegressor(random_state=0)

In [377]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
regressor.fit(X_train, y_train)

# Predict and evaluate
y_pred = regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
score_train = regressor.score(X_train, y_train)


print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')
print(f'Score train: {score_train}')

Mean Absolute Error: 87336.00844715665
R-squared: 0.7795768976211548
Score train: 0.9187541007995605


In [378]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRFRegressor
from sklearn.datasets import make_regression

X = df.drop(columns=["Price"], axis = 1)
y = df["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7, test_size= 0.2)


# Initialize the model with the best parameters
regressor = xgb.XGBRFRegressor(max_depth=50)

# cross_val_scores = cross_val_score(regressor, X, y, cv=10)
# # Print the cross-validation scores for each fold
# print("Cross-validation scores for each fold:", cross_val_scores)
# # Print the mean cross-validation score
# print("Mean cross-validation score:", cross_val_scores.mean())

# Train the model
regressor.fit(X_train, y_train, eval_set = [(X_test, y_test)], verbose = True)

# Predict and evaluate
y_pred = regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
score_train = regressor.score(X_train, y_train)


print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')
print(f'Score train: {score_train}')

[0]	validation_0-rmse:206998.95377
Mean Absolute Error: 79244.44968690092
R-squared: 0.7867562770843506
Score train: 0.9907063245773315


In [382]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

# Prepare the data
X = df.drop(columns=["Price"], axis=1)
y = df["Price"]


# Initialize the regressor
regressor = RandomForestRegressor(random_state=0, n_jobs=-1)

# Perform cross-validation
cross_val_scores = cross_val_score(regressor, X, y, cv=10)

# Print the cross-validation scores for each fold
print("Cross-validation scores for each fold:", cross_val_scores)

# Print the mean cross-validation score
print("Mean cross-validation score:", cross_val_scores.mean())

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
regressor.fit(X_train, y_train)

# Predict and evaluate
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
score_train = regressor.score(X_train, y_train)
score_test = regressor.score(X_test, y_test)

print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')
print(f'Score train: {score_train}')
print(f'Score test: {score_test}')


Cross-validation scores for each fold: [0.82119668 0.77941024 0.72666525 0.81205142 0.80155497 0.81135844
 0.80256396 0.76320163 0.76527555 0.76567496]
Mean cross-validation score: 0.7848953115931596
Mean Absolute Error: 78474.25173506683
R-squared: 0.8003844739504511
Score train: 0.9700837460724501
Score test: 0.8003844739504511


In [380]:
len(df)

83379